Scrapes the novelupdates webpage, http://www.novelupdates.com/
for novel information and cleans it.
Finally saves the dataset as a csv file.

Didn't find any easy way to get all the novel ids, hence, the novels listing is first used to get all ids.

In [37]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

novels_page = "http://www.novelupdates.com/novelslisting/?st=1&pg="

In [40]:
# Get the maximum number of novel pages from the listings

def get_novel_list_max_pages(page):
    soup = BeautifulSoup(page.content, 'html.parser')
    dig_pag = soup.find('div', attrs={'class':'digg_pagination'})
    page_links = dig_pag.find_all('a')
    last_page_link = str(page_links[2]) # The last page is the 3rd
    num = re.search('pg=\d+', last_page_link).group()[3:]
    return int(num)

page = requests.get(novels_page + '1')
novels_max_pages = get_novel_list_max_pages(page)

In [ ]:
# Get all novel ids from the novels listing

# Overwrite for testing
# novels_max_pages = 3 

def get_novel_ids(page):
    soup = BeautifulSoup(page.content, 'html.parser')
    table = soup.find('table', attrs={'id':'myTable'})
    table_body = table.find('tbody')
    rows = table_body.find_all('tr')
    
    novel_ids = []
    for row in rows:
        col = row.find_all('td')[-1]
        novel_id = col.a['id'][3:]
        novel_ids.append(novel_id)
    return novel_ids

all_novel_ids = []
for i in range(1,novels_max_pages+1):
    page = requests.get(novels_page + str(i))
    novel_ids = get_novel_ids(page)
    all_novel_ids.extend(novel_ids)
    time.sleep(1)

df = pd.DataFrame(all_novel_ids)
df.to_csv("novel_ids.csv")